In [1]:
%pwd
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import csv

labels = [-1, 'CN', 'EMCI', 'LMCI', 'AD']

In [2]:
#Loading in data
df = pd.read_csv('Data/ADNI_Gene_Expression_Profile.csv', header=2)
df = df[6:]
df = df.drop(df.columns[[1, -1]], axis=1)
df = df.rename({'SubjectID':'ProbeSet', 'Unnamed: 2':'Gene'}, axis=1)
df = df.set_index(['Gene', 'ProbeSet'])
df = df.loc[df.index.dropna()]

dfanno = pd.read_csv('Data/ADNIMERGE.csv')
dfanno = pd.Series(dfanno['DX_bl'].values, index=dfanno['PTID'])
dfanno = dfanno.groupby(dfanno.index).last()
dfanno = dfanno.loc[df.columns]

C:\Users\andre\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (329,480) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\andre\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Load Genes
with open("genes.txt", "rb") as fp:   # Unpickling
    genes = pickle.load(fp)

In [4]:
#Choose Genes
dfanno = dfanno.loc[df.columns]
dftop = df.loc[genes]
dftop = dftop.groupby(dftop.index).first()
dftop = dftop.astype(float)

In [5]:
#Standardize
dftop = dftop.astype(float)
dftop = dftop.sub(dftop.mean(1), axis=0).div(dftop.std(1), axis=0)
X = dftop.T
y = dfanno

In [6]:
AD = y.replace('AD', 4).replace('LMCI', 3).replace('EMCI', 2).replace('CN', 1)

In [7]:
BINARY = y.replace('AD', 1).replace('LMCI', 0).replace('EMCI', 0).replace('CN', 0)

In [8]:
X.columns = ['_'.join(i) for i in X]

In [9]:
snp = pd.read_csv('C:/Users/andre/OneDrive/Documents/Science Fair/GWAS/WGS_Omni2.5M_20140220/SNPArray.raw', sep='\s{1,}', index_col=1)
snpanno = pd.read_csv('Data/TETRADdiagnosis.csv', header=None)

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [10]:
snp = snp.drop(snp.columns[[0, 1, 2, 3, 4]], axis=1).astype(pd.Int32Dtype()).loc[snpanno[0]]

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [11]:
tetdf = pd.concat([X, AD], axis=1).rename(columns={0: "Diagnosis"})

In [12]:
tetdf2 = pd.concat([X, snp, AD], axis=1).rename(columns={0: "Diagnosis"})

In [13]:
tetdf_bin = pd.concat([X, BINARY], axis=1).rename(columns={0: "Diagnosis"})

In [14]:
tetdf_bin2 = pd.concat([X, snp, BINARY], axis=1).rename(columns={0: "Diagnosis"})

In [15]:
snp.to_csv('Data/TETRADsnp.csv', quoting=csv.QUOTE_MINIMAL)

In [16]:
X.to_csv('Data/TETRADgene.csv', quoting=csv.QUOTE_MINIMAL)

In [17]:
AD.to_csv('Data/TETRADdiagnosis.csv')

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [18]:
tetdf.to_csv('Data/TETRADfull.csv', quoting=csv.QUOTE_MINIMAL)

In [19]:
tetdf2.to_csv('Data/TETRADfull2.csv', quoting=csv.QUOTE_MINIMAL)

In [20]:
tetdf_bin.to_csv('Data/TETRADfull_binary.csv', quoting=csv.QUOTE_MINIMAL)

In [21]:
tetdf_bin2.to_csv('Data/TETRADfull_binary2.csv', quoting=csv.QUOTE_MINIMAL)